Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  # Sample the dataset, if you have insufficient memory
  train_dataset = train_dataset[:train_dataset.shape[0]//5,:,:]
  train_labels = train_labels[:train_labels.shape[0]//5]
  valid_dataset = valid_dataset[:valid_dataset.shape[0]//5,:,:]
  valid_labels = valid_labels[:valid_labels.shape[0]//5]
  # display the size
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (70548, 28, 28) (70548,)
Validation set (35274, 28, 28) (35274,)
Test set (18724, 28, 28) (18724,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (70548, 28, 28, 1) (70548, 10)
Validation set (35274, 28, 28, 1) (35274, 10)
Test set (18724, 28, 28, 1) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

---
reference for conv2d function:  
https://www.tensorflow.org/versions/v0.6.0/api_docs/python/nn.html#conv2d  
http://stackoverflow.com/questions/34619177/what-does-tf-nn-conv2d-do-in-tensorflow
#### `tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, name=None)`
- `input`: tensor of shape `[batch, in_height, in_width, in_channels]` where the input data resides
- `filter(kernel)`: tensor of `[filter_height, filter_width, in_channels, out_channels]` where the weights to be trained resides
- `strides`: A list of ints. 1-D of length 4. The stride of the sliding window for each dimension of input. For the most common case of the same horizontal and vertices strides, `strides = [1, stride, stride, 1]`
- `padding`: A string from: `"SAME", "VALID"`. The type of padding algorithm to use.

----
#### what conv2d does: * refer to the CNN1.jpg, CNN2.jpg in the Github repo
1. Flattens the filter to a 2-D matrix with shape `[filter_height * filter_width * in_channels, output_channels]`  
`filter_height * filter_width * in_channels` is the size of a <b>single</b> filter on a <b>single</b> image, and `output_channels` is the (number of filters) = (depth)
2. Extracts image patches from the the input tensor to form a virtual tensor of shape `[batch, out_height, out_width, filter_height * filter_width * in_channels]`. Each image patches extracted is flattened to have the size of `[filter_height * filter_width * in_channels]`. `out_height` and `out_width` is determined by the size of stride and the type of padding and `out_height*out_width` is the total number of image patches extracted from the input image
3. For each patch, right-multiplies the filter matrix and the image patch vector. i.e. image patch(vector of size `filter_height * filter_width * in_channels`) `*` `filter`(matrix of size [`filter_height * filter_width * in_channels,out_channels]`). Information of each patch is compressed by the dot product between each patch & each filter column in the filter matrix.
----
reference for the CNN: http://deeplearning.net/tutorial/lenet.html

In [5]:
batch_size = 16
patch_size = 5 # convolution filter width & height
depth = 16
num_hidden = 64 # number of hidden relu layers

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size / 4 * image_size / 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases) # conv + layer1_biases is the logit
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    # flatten the tensor into 2-d matrix to run a MLP classifier
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]]) 
    # shape[0]: batch size(index of an image), shape[1] * shape[2] * shape[3]: compressed data vector of the image
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) # fully-connected network
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation, using fully-connected network
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 3.4212
Minibatch accuracy: 6.2%
Validation accuracy: 10.1%
Minibatch loss at step 1000 : 0.254464
Minibatch accuracy: 93.8%
Validation accuracy: 83.2%
Minibatch loss at step 2000 : 0.388922
Minibatch accuracy: 87.5%
Validation accuracy: 84.5%
Minibatch loss at step 3000 : 0.791278
Minibatch accuracy: 81.2%
Validation accuracy: 85.9%
Minibatch loss at step 4000 : 0.469344
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 5000 : 0.318295
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Minibatch loss at step 6000 : 0.320415
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 7000 : 0.626232
Minibatch accuracy: 81.2%
Validation accuracy: 87.8%
Minibatch loss at step 8000 : 0.682507
Minibatch accuracy: 81.2%
Validation accuracy: 87.5%
Minibatch loss at step 9000 : 0.525274
Minibatch accuracy: 81.2%
Validation accuracy: 88.1%
Minibatch loss at step 10000 : 0.374011
Minibatch accuracy: 93.8%
Validati

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality.(halved) Replace the strides a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

# Don't Run the codes below. Severe Memory Leak in my Notebook PC

In [5]:
batch_size = 16
patch_size = 5 # convolution filter width & height
depth = 16
num_hidden = 64 # number of hidden relu layers

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size / 4 * image_size / 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1,1,1,1], padding='SAME') # note that the conv2d itself has stride of 1
    hidden = tf.nn.relu(conv + layer1_biases) # conv + layer1_biases is the logit
    pool = tf.nn.max_pool(value=hidden, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # max-pooling
    conv = tf.nn.conv2d(pool, layer2_weights, [1,1,1,1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(value=hidden, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    # flatten the tensor into 2-d matrix to run a MLP classifier
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]]) 
    # shape[0]: batch size(index of an image), shape[1] * shape[2] * shape[3]: compressed data vector of the image
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) # fully-connected network
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation, using fully-connected network
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [ ]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---